                        FINDING THE lEADING AND LAGGING PEAKS

Approaches

1) Clustering the Data Points
2) Defining a box of relavent data and filter out unnecessary points

THE BOX OF RELEVANCY

How do we store this info? 
   
1) (a, b) start and end indexes of the box
2) Just store an array of larger peaks and itterate through them
3) Start iterrating the dataframe rows and start a loop between two high points
    
    a) In this space, itteratively eliminate the noise from bottom up
    
    b) SO we are deleting the noise and accessing the length of the droplets

To my understanding there are two main factors in deciding if a peak is the lowest peak (lagging/leading)

1) How higher up it is in comparision to the other points? (prominance)

2) How close is it to the next higher peak? (so we choose the last spike and not confuse it for the encapsulated ssc)

3) Even though each droplet can change in size, we can still find a radius within which all the approximated lengths fall. Maybe turn this into a criteria for confirming the future itterations? Like a droplet range that upgrades while the algorithm goes through the dataset. That way, we can set a limit to how far this droplet radius can go.

CLUSTTERING THE DATA

I have a diffirent use for clusterring:

--> Between two higher peaks, if the data between them is clusterred, the specific peak signifying the end of the droplet will mostly be isolated. SO we can cluster up and the irrelevant data and delete it.

Ideas : Using the Destiny Based Clusterring algorithm to classify the peaks?

???? Do we need ML here ????

In [14]:
#Setting up the dataframe

import pandas as pd
import numpy as np
from scipy import signal
import os
import shutil

#Rewritting the text files to CSV's
def set_path(name):
    default_path = "/home/sweedy/Desktop/MFA GUI/Datasets/"
    #If we find a tsv or csv file
    if name.endswith('.tsv' or '.csv'):
        return
    #If it's a folder
    else : 
        #Convert text files to csv
        folder_path = default_path + name
        for file in os.listdir(folder_path):
            #Make sure you get to the root directory
            while True:
                file_path = os.path.join(folder_path, file)
                if not os.path.isdir(file_path):
                    break
                else :
                    os.path.join(folder_path, file_path)
            #To make sure csv files aren't renamed
            if file_path.endswith('.csv' or '.tsv'):
                continue
            data = pd.read_csv(file_path)
            data.to_csv(file_path + ".csv", index = None)
            os.remove(file_path)
        return

set_path("dg2.tsv")


In [15]:
def set_dataframe(path, seperator):
    df = pd.read_csv(path, sep = seperator)
    means = []
    cols = list(df)
    print(cols)
    for c in cols:
        mean = df[c].mean()
        if mean == 0:
            df.drop(c, axis=1, inplace=True)
        means.append(mean)
    print(means)
    #deciding which column is the Ssc
    df.columns = means
    max_ = max(df.columns)
    df.rename(columns = {max_ : (str(round(max_, 2)) + '_Ssc')}, inplace = True)

    #dRenaming the rest with Fl
    means.remove(max(means))
    df.rename(columns = {m : str(round(m, 2)) + '_Fl' for m in means}, inplace = True)
    df = df.dropna()
    #Set the index as columns
    df.reset_index(inplace=True)
    df['index'] = df.index

    return df.iloc[:20000]

df = set_dataframe("/home/sweedy/Desktop/MFA GUI/Datasets/dg4 _half laser_final data.tsv", r"\t")

print(df)

/home/sweedy/anaconda3/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



['6', '33']
[32.8032099876789, 173.13119662454903]
       index  32.8_Fl  173.13_Ssc
0          0       24          42
1          1       17          37
2          2       20          41
3          3       14          36
4          4       22          43
...      ...      ...         ...
19995  19995       20          39
19996  19996       25          30
19997  19997       23          19
19998  19998       23          45
19999  19999       21          48

[20000 rows x 3 columns]


    I'm making the assumptions :

1) The highest peaks in the dataset are those beyond atleast half the highest point

2) Minimum Distance between the peaks should atleast be a hundred clicks apart?


In [16]:
#setting up the peaks function for the highest data points
ssc_col = list(df.filter(regex='_Ssc$',axis=1))[0]
ssc = df[ssc_col]

min_height = int(max(ssc)/2)
print("Setting Height at : ", min_height)
print("Setting Distance at : ", 100)
high_peaks, _ = signal.find_peaks(ssc, height=min_height, distance=100)

Setting Height at :  965
Setting Distance at :  100


In [17]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.line(df, x='index', y = ssc_col)
fig.update_traces(line_color = 'rgb(72,61,139)')
fig.add_trace(go.Scatter(x = high_peaks, y = ssc.iloc[high_peaks], mode = 'markers'))
fig.update_layout({
    "showlegend" : True
})
fig.show()
print("Approximate number of droplets based on leading peak count: ", len(high_peaks))

Approximate number of droplets based on leading peak count:  157


SO.... I'll try setting a default parameter here and see if that gives me a majority of the smaller peaks. That paramter is not being set in the peak calling but in defining the box dimention. 

We make the box smaller by removing a margin of 'n' points to avoid recognizing the encapsulated SSC.

In [18]:
lagging_peaks = []
print(high_peaks)
for n in range(len(high_peaks)):
    #Skip the last high peak as it would have already been acessed
    if high_peaks[n] == high_peaks[-1]:
        break
    #store the box of info
    print(high_peaks[n])
    print(high_peaks[n+1])
    box = df.iloc[(high_peaks[n]+30) : (high_peaks[n+1]-30)]
    print(box)
    #find high peaks (manually setting params for now)

    peaks, _ = signal.find_peaks(box[ssc_col])
    print("Peaks found in box : ", peaks)
    #picking the largest value found by the peaks function

    peaks = list(peaks)
    lag_peak = max(box[ssc_col].iloc[peaks])
    lag_peak_idx = list(box[ssc_col]).index(lag_peak)
    lag_peak_real_idx = box['index'].iloc[lag_peak_idx]
    print("The lagging peak : ", lag_peak_real_idx)
    lagging_peaks.append(lag_peak_real_idx)
    del box

print("The list of lagging peaks are : ", lagging_peaks)


[   45   178   307   437   572   700   831   962  1092  1221  1346  1467
  1587  1708  1831  1953  2073  2194  2315  2452  2581  2708  2838  2967
  3097  3224  3353  3481  3609  3736  3864  3990  4118  4248  4378  4509
  4640  4767  4895  5027  5154  5283  5412  5539  5669  5798  5926  6053
  6181  6309  6439  6567  6694  6820  6947  7073  7199  7325  7452  7579
  7701  7822  7945  8068  8190  8311  8433  8557  8682  8808  8934  9059
  9187  9316  9443  9571  9698  9827 10025 10152 10278 10405 10531 10657
 10784 10905 11026 11148 11270 11392 11513 11635 11759 11885 12011 12136
 12262 12388 12516 12642 12767 12893 13019 13143 13269 13396 13522 13649
 13773 13890 14006 14125 14244 14365 14487 14613 14734 14856 14979 15101
 15226 15353 15479 15604 15730 15857 15985 16113 16243 16371 16500 16629
 16756 16882 17007 17130 17254 17380 17535 17665 17796 17929 18061 18193
 18326 18459 18592 18726 18860 18994 19129 19265 19398 19532 19665 19796
 19928]
45
178
     index  32.8_Fl  173.13_Ssc
75  

In [19]:
fig_lag = px.line(df, x='index', y = ssc_col)
fig_lag.update_traces(line_color = 'rgb(72,61,139)')
fig_lag.add_trace(go.Scatter(x = high_peaks, y = df[ssc_col].iloc[high_peaks], mode = 'markers'))
fig_lag.add_trace(go.Scatter(x = lagging_peaks, y = df[ssc_col].iloc[lagging_peaks], mode = 'markers'))
fig_lag.update_layout({
    "showlegend" : True
})
fig_lag.show()

Okay this seems to work, but there should some automated way to box in the data without having to set the default value for each box.

I'll try taking the mid point between two peaks and then drawing a radius to enclose 
2/4th's of the boxed in data between each set of taller peaks. Thus the radius will be 
1/4th the data

Note : We're drawing out this radius to eliminate the encapsulated SSC from being recognized.

Once we have the lagging peaks, we can find all peaks and consider the high peak calculated in the box, excluding the lagging peak. That should let us derive the encapsulated SSC

In [20]:
#New boxing method by radius
new_lagging_peaks = []

for n in range(len(high_peaks)):
    #Skip the last high peak as it would have already been acessed
    if high_peaks[n] == high_peaks[-1]:
        break
    #store the box of info
    mid = int((high_peaks[n+1] - high_peaks[n])/2)
    radius = int(mid/2)
    box = df.iloc[(high_peaks[n]+radius) : (high_peaks[n+1]-radius)]
    print(radius)
    print(box)

    #find high peaks (manually setting params for now)
    peaks, _ = signal.find_peaks(box[ssc_col])
    print("Peaks found in box : ", peaks)
    #picking the largest value found by the peaks function

    peaks = list(peaks)
    new_lag_peak = max(box[ssc_col].iloc[peaks])
    new_lag_peak_idx = list(box[ssc_col]).index(new_lag_peak)
    new_lag_peak_real_idx = box['index'].iloc[new_lag_peak_idx]
    print("The lagging peak : ", new_lag_peak_real_idx)
    new_lagging_peaks.append(new_lag_peak_real_idx)
    del box

print(new_lagging_peaks)

33
     index  32.8_Fl  173.13_Ssc
78      78       25          32
79      79       18          31
80      80       13          36
81      81       14          30
82      82       18          21
..     ...      ...         ...
140    140       26          37
141    141       20          33
142    142       18          37
143    143       21          30
144    144       25          27

[67 rows x 3 columns]
Peaks found in box :  [ 2  5 11 13 17 22 25 28 34 37 39 41 43 47 50 52 55 58 60 62 64]
The lagging peak :  91
32
     index  32.8_Fl  173.13_Ssc
210    210       18          30
211    211       14          32
212    212       11          32
213    213       20          35
214    214       28          21
..     ...      ...         ...
270    270       23          33
271    271       28          27
272    272       22          36
273    273       22          33
274    274       16          33

[65 rows x 3 columns]
Peaks found in box :  [ 3  5  9 14 18 20 27 29 33 35 40 44 50 54 57 60

In [21]:
new_fig_lag = px.line(df, x='index', y = ssc_col)
new_fig_lag.update_traces(line_color = 'rgb(72,61,139)')
new_fig_lag.add_trace(go.Scatter(x = high_peaks, y = df[ssc_col].iloc[high_peaks], mode = 'markers'))
new_fig_lag.add_trace(go.Scatter(x = lagging_peaks, y = df[ssc_col].iloc[lagging_peaks], mode = 'markers'))
new_fig_lag.update_layout({
    "showlegend" : True
})
new_fig_lag.show()

Datasets to test :

1)dg2.tsv

2)dg4 _half laser_final data.tsv

3)peak_2.tsv

4)stained cells (with SSC) - 22nd Oct
 **folder**

5)The one with multiplexed PE and Rho123 - December 19 to Jan first week 20
 **folder**

6)The one with encapsulated cells - 18th October 20
 **folder**